# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [10]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = "https://country-leaders.onrender.com/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print("Everything is ok")
else:
    print(f"Error on {req.status_code}")

Everything is ok


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [11]:
# Set the countries_url variable (1 line)
country_url = "https://country-leaders.onrender.com/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req_countries = requests.get(f"{root_url}/countries")
# Get the JSON content and store it in the countries variable (1 line)
req_countries.json()
# display the request's status code and the countries variable (1 line)
print(f"{req_countries.status_code}, req_countries")

403, req_countries


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [12]:
# Set the cookie_url variable (1 line)
url_cookie = "https://country-leaders.onrender.com/cookie"
# Query the enpoint, set the cookies variable and display it (2 lines)
req_cookie = requests.get(url_cookie)
print(req_cookie.cookies)

<RequestsCookieJar[<Cookie user_cookie=235248e7-f91f-4814-bb42-f45ccc2888a6 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [14]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
import requests
countries = requests.get(country_url, cookies=req_cookie.cookies).json()

# display the countries variable (1 line)
print(countries)

['ma', 'us', 'ru', 'fr', 'be']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [ ]:
# Set the leaders_url variable (1 line)
url_leaders = "https://country-leaders.onrender.com/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = req_leaders = requests.get(url_leaders, cookies=req_cookie.cookies).json()
# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [ ]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leader_fr = requests.get(url_leaders, cookies=req_cookie.cookies, params={"country": "fr"}).json()

# display the leaders variable (1 line)
print(leader_fr)

[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
leader_per_country = []
for count in countries:    
    leader_per_country.append(requests.get(url_leaders, cookies=req_cookie.cookies, params={"country":count}).json())

print(leader_per_country)


[[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': '

In [ ]:
# or 1 line
for count in countries: leader_per_country.append(requests.get(url_leaders, cookies=req_cookie.cookies, params={"country":count}).json())
print(leader_per_country)

[[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': '

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [206]:
# < 15 lines
import requests

def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookie_url = "https://country-leaders.onrender.com/cookie"
    leaders_url = "https://country-leaders.onrender.com/leaders"
    #requête du cookie
    cookies = requests.get(cookie_url)
    cookies = cookies.cookies
    print(f"This is the cookie: {cookies}.")

    #crée la liste pays
    countries = requests.get(f"{root_url}/countries", cookies=cookies).json()
    print(f"The countries are: {countries}.")

    #choper les leaders (boucle)
    leader_per_country = {}
    for country in countries:
        leader_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country":country}).json()
    return leader_per_country

get_leaders()


This is the cookie: <RequestsCookieJar[<Cookie user_cookie=aaac2139-5ce5-4125-a578-2464e0d0e308 for country-leaders.onrender.com/>]>.
The countries are: ['be', 'us', 'ru', 'fr', 'ma'].


{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate': '2009-11-25',
   'end_mandate': '2011-12-06'},
  {'id': 'Q12983',
   'first_name': 'Herman',
   'last_name': 'None',
   'birth_date': '1947-10-31',
   'death_date': None,
   'place_of_birth': 'Etterbeek',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
   'start_mandate': '2008-12-30',
   'end_mandate': '2009-11-25'},
  {'id': 'Q14989',
   'first_name': 'Léon',
   'last_name': 'Delacroix',
   'birth_date': '1867-12-27',
   'd

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [62]:
# 2 lines
leaders_per_country = get_leaders()
for leader in leaders_per_country:
    print(leaders_per_country[leader])

This is the cookie: <RequestsCookieJar[<Cookie user_cookie=70d04021-5416-4e6d-a3dd-c0634261e733 for country-leaders.onrender.com/>]>.
The countries are: ['ma', 'us', 'ru', 'fr', 'be'].
[{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [131]:
# 3 lines
def get_text():
    root_url = "https://country-leaders.onrender.com"
    cookie_url = "https://country-leaders.onrender.com/cookie"
    leaders_url = "https://country-leaders.onrender.com/leaders"
    cookies = requests.get(cookie_url)
    cookies = cookies.cookies

    leaders_fr = requests.get(leaders_url, cookies=cookies, params={"country":"fr"}).json()
    #print(leaders_fr)

    url_list = []
    for elem in leaders_fr:
        url_list.append(elem["wikipedia_url"])
        #print(url_list)

    wikipedia_display = requests.get(url_list[0])
    return wikipedia_display.text
        
get_text()

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>François Hollande — Wikipédia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled ve

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
# 3 lines
from bs4 import BeautifulSoup
import requests

html_doc = get_text()
soup = BeautifulSoup(html_doc, "html.parser")

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   François Hollande — Wikipédia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-conte

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [167]:
# 2 lines
content = soup.select("#mw-content-text > div.mw-parser-output > p")



If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [174]:
# <10 lines
paragraphs = []

for p in content:
    if len(p.getText(strip=True)) > 0:
        paragraphs.append(p)
    else:
        continue

first_paragraph = paragraphs[0]
print(first_paragraph.getText(strip=True))

François Hollande[fʁɑ̃swaɔlɑ̃d][n 3]Écouterⓘ, né le12 août 1954àRouen(Seine-Inférieure), est unhaut fonctionnaireethomme d'Étatfrançais. Il estprésident de la République françaisedu15 mai 2012au14 mai 2017.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [ ]:
# 10 lines
import json
import requests
from bs4 import BeautifulSoup

def get_first_paragraph(wikipedia_url: str):
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.text, "html.parser")
    content = soup.select("#mw-content-text > div.mw-parser-output > p")
    paragraphs = []
    for p in content:
        if len(p.getText(strip=True)) > 0:
            paragraphs.append(p)
        else:
            continue
    first_paragraph = paragraphs[0]
    return first_paragraph.getText(strip=True)

get_first_paragraph("https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande")
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

"François Hollande[fʁɑ̃swaɔlɑ̃d][n 3]Écouterⓘ, né le12 août 1954àRouen(Seine-Inférieure), est unhaut fonctionnaireethomme d'Étatfrançais. Il estprésident de la République françaisedu15 mai 2012au14 mai 2017."

### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [202]:
# 3 lines
import re

patterns = [r"/[^/]*/\s*;\s*", r"r\/.*\/", r"\[.*?\]", r"ⓘ"]

first_paragraph = get_first_paragraph("https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande")
for pattern in patterns:
    first_paragraph = re.sub(pattern, "", first_paragraph)

print(first_paragraph.strip())


François HollandeÉcouter, né le12 août 1954àRouen(Seine-Inférieure), est unhaut fonctionnaireethomme d'Étatfrançais. Il estprésident de la République françaisedu15 mai 2012au14 mai 2017.


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [203]:
# 10 lines
import requests
from bs4 import BeautifulSoup
import re

def get_first_paragraph(wikipedia_url: str):
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.text, "html.parser")
    content = soup.select("#mw-content-text > div.mw-parser-output > p")    
    paragraphs = []
    for p in content:
        if len(p.getText(strip=True)) > 0:
            paragraphs.append(p)
        else:
            continue
    first_paragraph = paragraphs[0].get_text(strip=True)
    
    patterns = [r"/[^/]*/\s*;\s*", r"r\/.*\/", r"\[.*?\]", r"ⓘ"]
    
    for pattern in patterns:
        first_paragraph = re.sub(pattern, "", first_paragraph)

    return first_paragraph.strip()

# Test
print(get_first_paragraph("https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande"))

François HollandeÉcouter, né le12 août 1954àRouen(Seine-Inférieure), est unhaut fonctionnaireethomme d'Étatfrançais. Il estprésident de la République françaisedu15 mai 2012au14 mai 2017.


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [205]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [240]:

import requests
from bs4 import BeautifulSoup
import re

root_url = "https://country-leaders.onrender.com"
cookie_url = "https://country-leaders.onrender.com/cookie"
leaders_url = "https://country-leaders.onrender.com/leaders"
countries_url = "https://country-leaders.onrender.com/countries"

cookies = requests.get(cookie_url).cookies


def get_leaders():
    countries = requests.get(countries_url, cookies=cookies).json()
    #print(f"The countries are: {countries}.")

    leader_per_country = {}
    for country in countries:
        leader_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country":country}).json()
    return leader_per_country

get_leaders()


def get_first_paragraph(wikipedia_url: str):
    req = requests.get(wikipedia_url)
    soup = BeautifulSoup(req.text, "html.parser")
    content = soup.select("#mw-content-text > div.mw-parser-output > p")    
    paragraphs = []
    for p in content:
        if len(p.getText(strip=True)) > 0:
            paragraphs.append(p)
        else:
            continue
    first_paragraph = paragraphs[0].get_text(strip=True)
    
    patterns = [r"/[^/]*/\s*;\s*", r"r\/.*\/", r"\[.*?\]", r"ⓘ"]
    
    for pattern in patterns:
        first_paragraph = re.sub(pattern, "", first_paragraph)

    return first_paragraph.strip()

def get_first_paragraph_per_country():
    leaders_per_country = get_leaders()
    first_paragraphs = {}

    first_leader = []
    for country, leaders in leaders_per_country.items():
        first_leader = next((l for l in leaders if l.get("wikipedia_url")), None)

        for leader in first_leader:
            url = first_leader["wikipedia_url"]
            paragraph = get_first_paragraph(url)
            first_paragraphs[country] = {country}, {first_leader.get("first_name")}, {first_leader.get("last_name")}, paragraph


    return first_paragraphs

print(get_first_paragraph_per_country())


{'fr': ({'fr'}, {'François'}, {'Hollande'}, "François HollandeÉcouter, né le12 août 1954àRouen(Seine-Inférieure), est unhaut fonctionnaireethomme d'Étatfrançais. Il estprésident de la République françaisedu15 mai 2012au14 mai 2017."), 'be': ({'be'}, {'Guy'}, {'Verhofstadt'}, 'Guy Maurice Marie-Louise Verhofstadt(uitspraak) (Dendermonde,11 april1953) is eenBelgischpoliticusvoor deOpen Vlaamse Liberalen en Democraten(Open Vld). Hij waspremier van Belgiëvan 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in hetEuropees Parlement, waar hij van 2009 tot 2019 fractieleider van deAlliantie van Liberalen en Democraten voor Europa(ALDE) was.'), 'ma': ({'ma'}, {'Mohammed'}, {'None'}, 'مُحمد السادس بن الحسن الثاني العلوي(مواليد 21 أغسطس 1963) هو ملك المملكة المغربية منذ عام 1999 والملك الثالث والعشرون للمغرب منسلالة العلويين الفيلاليين، تولى الحكم خلفًا لوالده الملكالحسن الثانيبعد وفاته، وبويع ملكًا يوم الجمعة 9 ربيع الثاني سنة 1420 هـ الموافق 23 

In [242]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines



Check the output of your function again.

In [23]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [24]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
# <25 lines


Test your new functions.



## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [26]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [27]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [28]:
# 3 lines


In [29]:
# Call the function (1 line)


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!